# Searchlight

## Imports

In [31]:
import pandas as pd
import re
import numpy as np
from datascience import *
import urllib
from selenium import webdriver
from time import sleep
import requests
from bs4 import BeautifulSoup as Soup
import os

## Initialize DataScience Tables

In [32]:
#Speech Table
speeches = Table().with_columns("speech_id", make_array(), 
                                "speaker_id", make_array(), 
                                "proceeding_id", make_array(), 
                                "topic_id", make_array(), 
                                "word_count", make_array(), 
                                "speech_text", make_array(),
                                'file_name', make_array())

In [33]:
speeches

speech_id,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name


In [34]:
#Speaker Table
speakers = Table().with_columns("speaker_id", make_array(), 
                                "first_name", make_array(), 
                                "last_name", make_array(), 
                                "type" , make_array(),
                                "party", make_array(), 
                                "state", make_array(), 
                                "district", make_array(),
                                "bio_guide_id", make_array(),
                                "congress_id", make_array())

In [35]:
speakers

speaker_id,first_name,last_name,type,party,state,district,bio_guide_id,congress_id


In [36]:
topics = Table().with_columns("topic_id", make_array(), 
                                "topic_name", make_array())

In [37]:
topics

topic_id,topic_name


In [38]:
#Proceedings Table
proceedings = Table().with_columns("proceeding_id", make_array(), 
                              "date", make_array(),
                              "title", make_array())

In [39]:
proceedings

proceeding_id,date,title


## Initializing Parsing

### Text Parsing

In [40]:
def remove_space(regex):
    return regex.group().replace(' ', '')

In [41]:
def sep_speech(string):
    parse_file = ''
    with open(string) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('\n', '')
#     parse_file = parse_file.replace('Mr. President', 'MrPresident')
#     parse_file = parse_file.replace('Mr. Short', 'Mr.Short')
    parse_file = re.sub('Mr. [A-Z][a-z]', remove_space, parse_file)
    
    split = re.split(r'Mr. |Ms. |Mrs. ', parse_file)
    split.pop(0)
    name_and_speech = make_array()
    for i in np.arange(len(split)):
        try:
            lastname = re.match('[A-Z]*\. ', split[i]).group(0)[:-2]
            name_and_speech = np.append(name_and_speech, lastname)
            value = re.sub('[A-Z]\w*\. ', '', split[i])
            name_and_speech = np.append(name_and_speech, value)
        except:
            abcabcabc = 1
    return name_and_speech

In [42]:
def sep_date_from_file(file):
    abcdef = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', file)
    return re.split('-', abcdef[0])

In [43]:
def find_title(file_name):
    parse_file = ''
    with open(string) as file:
        for line in file:
            parse_file += line
    parse_file = parse_file.replace('Mr. President', 'MrPresident')
    title = re.findall('[A-Z \'-]+[A-Z0-9-,\. ]*[Continued]*\\n', parse_file)
    return title[0].strip()

### Mods Parsing

In [44]:
def getAllExtensions(file):
    handler = open(file).read()
    soup = Soup(handler, "lxml")
    return soup.find_all('extension')

In [45]:
master_extensions = getAllExtensions("mastermods.xml")

In [46]:
def getCongMemberExtension(extensions, last_name):
    for extension in extensions:
        ext = str(extension)
        if last_name in ext:
            return extension

In [47]:
def getCongMemberExtensionFromFile(last_name, filename):
    handler = open(filename).read()
    soup = Soup(handler, "lxml")
    extensions = soup.find_all('extension')
    for extension in extensions:
        ext = str(extension)
        if last_name in ext:
            return extension

In [48]:
def getCongMemberTag(congMemberExtension):
    contents = congMemberExtension.contents
    for tag in contents:
        tag_str = str(tag)
        if 'congmember' in tag_str:
            return tag

In [49]:
def getParty(congMemberTag):
    return congMemberTag.attrs['party']
def getType(congMemberTag):
    return congMemberTag.attrs['type']
def getAuthorityId(congMemberTag):
    return congMemberTag.attrs['authorityid']
def getBioGuideId(congMemberTag):
    return congMemberTag.attrs['bioguideid']
def getState(congMemberTag):
    return congMemberTag.attrs['state']
def getCongressId(congMemberTag):
    return congMemberTag.attrs['congress']

In [50]:
def getDistrictTag(congMemberExtension):
    contents = congMemberExtension.contents
    for tag in contents:
        tag_str = str(tag)
        if 'district' in tag_str:
            return tag

In [51]:
def getFirstName(congMemberTag):
    contents = congMemberTag.contents
    name_tags = []
    for tag in contents:
        tag_str = str(tag)
        if 'name' in tag_str:
            name_tags += [tag]
    try:
        first_name = name_tags[1].string.split()[0]
    except:
        first_name = None
    return first_name

In [52]:
def getCongMemberInfoFromMaster(last_name, mods_filename):
    info = make_array()
    if last_name == 'BORDALLO' or last_name == 'CAPITO':
        return getCongMemberInfoFromLocal(last_name, mods_filename)
    try:
        extension = getCongMemberExtension(master_extensions, last_name)
        if extension is None:
            return getCongMemberInfoFromLocal(last_name, mods_filename)
        congMemberTag = getCongMemberTag(extension)
    except:
        return getCongMemberInfoFromLocal(last_name, mods_filename)
    
    
    
    congMemType = getType(congMemberTag)
    district = 'N/A'
    if congMemType == 'DELEGATE':
        try:
            info = getcongMemberInfoFromLocal(last_name, mods_filename)
        except:
            info = [99999999999999, 'First Name unavailable', last_name, congMemType, 'Party Info Unavailable','state info unavailable', district, 'BioGuideID unavailable', 'CongressID unavailable']
    else:
        if congMemType == 'REPRESENTATIVE':
            try:
                district_tag = getDistrictTag(extension)
                district = district_tag.string
            except:
                district = 'N/A'
    try:
        bioGuideID = getBioGuideId(congMemberTag)
    except:
        bioGuideID = 99999999999999999
    info = [getAuthorityId(congMemberTag), getFirstName(congMemberTag), last_name, congMemType, getParty(congMemberTag), getState(congMemberTag), district, bioGuideID, getCongressId(congMemberTag) ]
        
    return info

In [53]:
def getChamber(congMemberTag):
    return congMemberTag.attrs['chamber']

In [54]:
def getCongMemberInfoFromLocal(last_name, mods_filename):
    print('checking local file')
    extension = getCongMemberExtensionFromFile(last_name, mods_filename)
    info = make_array()
    congMemberTag = getCongMemberTag(extension)
    
    congMemChamber = getChamber(congMemberTag)
    congMemType = 'N/A'
    if congMemChamber == 'H':
        congMemType = 'REPRESENTATIVE'
    elif congMemChamber == 'S':
        congMemType = "SENATOR"
    
    district = 'N/A'
    if congMemType == 'REPRESENTATIVE':
        try:
            district_tag = getDistrictTag(extension)
            district = district_tag.string
        except:
            district = 'N/A'
            
#     info = np.append(info, getAuthorityId(congMemberTag))
#     info = np.append(info, getFirstName(congMemberTag))
#     info = np.append(info, last_name)
#     info = np.append(info, congMemType)
#     info = np.append(info, getParty(congMemberTag))
#     info = np.append(info, getState(congMemberTag))
#     info = np.append(info, district)
#     info = np.append(info, getBioGuideId(congMemberTag))
#     info = np.append(info, getCongressId(congMemberTag))
    try:
        authID = getAuthorityId(congMemberTag)
    except:
        authID = 99999999999999
    try:
        party = getParty(congMemberTag)
    except:
        party = 'Party information Unavailable'
    try:
        state = getState(congMemberTag)
    except:
        state = 'State Info Unavailable'
    try:
        bioID = getBioGuideId(congMemberTag)
    except:
        bioID = 9999999999999999
    info = [authID, getFirstName(congMemberTag), last_name, congMemType, party, state, district, bioID, getCongressId(congMemberTag) ]
    return info

## Parsing

In [64]:
#Populate the Speech Table
count = 0
for file in os.listdir("/Users/halliday/Desktop/scrape4"): #change directory once we have everything
    print(file)
    if file.endswith(".txt"):
        separated = sep_speech(file)
        i = 0
        while i < len(separated):
            row = make_array()
            text = separated[i+1]
            text = text.replace('MrPresident', 'Mr. President')
            if len(text) > 30:
                row = [count, separated[i], 'proceeding_id', 'topic-id', len(text.split()), text, file] 
                count += 1
                speeches = speeches.with_row(row)     
            i +=2
        count+= 1
        print('finished with file ', count)

CREC-2018-03-15-pt1-PgH1653-14.txt


FileNotFoundError: [Errno 2] No such file or directory: 'CREC-2018-03-15-pt1-PgH1653-14.txt'

In [235]:
speeches

speech_id,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name
48,CORNYN,proceeding_id,topic-id,2309,"Mr.President, many recall that Christmas came a little e ...",CREC-2018-01-03-pt2-PgS10.txt
49,DURBIN,proceeding_id,topic-id,25,"Mr.President, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS10.txt
51,DURBIN,proceeding_id,topic-id,893,"Mr.President, on September 5 of last year, Attorney Gene ...",CREC-2018-01-03-pt2-PgS11.txt
52,WHITEHOUSE,proceeding_id,topic-id,23,"Mr.President, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS11.txt
54,WHITEHOUSE,proceeding_id,topic-id,2035,"Thank you, Mr.President, and happy new year to you. For ...",CREC-2018-01-03-pt2-PgS13.txt
55,INHOFE,proceeding_id,topic-id,22,"Mr.President, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS13.txt
56,INHOFE,proceeding_id,topic-id,25,"Mr.President, I ask unanimous consent to speak in mornin ...",CREC-2018-01-03-pt2-PgS13.txt
57,INHOFE,proceeding_id,topic-id,1673,"Mr.President, I begin by wishing everyone a happy new ye ...",CREC-2018-01-03-pt2-PgS13.txt
59,INHOFE,proceeding_id,topic-id,1682,"Mr.President, I mentioned that there were three things I ...",CREC-2018-01-03-pt2-PgS16-2.txt
60,INHOFE,proceeding_id,topic-id,10,I yield the floor. The PRESIDING The Senator from ...,CREC-2018-01-03-pt2-PgS16-2.txt


In [236]:
speeches.where('speaker_id', are.equal_to('BORDALLO'))

speech_id,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name
698,BORDALLO,proceeding_id,topic-id,142,"Mr.Speaker, today I am proud to reintroduce my Compact I ...",CREC-2018-01-10-pt1-PgH91-7.txt


In [237]:
#create dictionairy of unique last names to files
distinct_lastname_table = speeches.group('speaker_id')
lastname_file_table = speeches.join('speaker_id', distinct_lastname_table, 'speaker_id')
lastname_file_table = lastname_file_table.drop('count').drop('speech_id').drop('proceeding_id').drop('topic_id').drop('word_count').drop('speech_text')
name_to_xml = {}
lastnames = lastname_file_table.column(0)
files = lastname_file_table.column(1)
count = 0
while count < len(lastnames):
    name_to_xml[lastnames[count]] = files[count].replace('.txt', '.xml')
    count += 1

In [238]:
#Populate Speaker Table

for name in list(name_to_xml.keys()):
    print(name)
    row = getCongMemberInfoFromMaster(name, name_to_xml[name])
    speakers = speakers.with_row(row)
speakers.show(5)
speeches.show(5)

ALEXANDER
ALLEN
AMASH
BABIN
BARRASSO
BEYER
BIGGS
checking local file
BILIRAKIS
BLUMENAUER
BLUMENTHAL
BLUNT
BOOKER
BOOZMAN
BORDALLO
checking local file
BOST
BROWN
BUCK
BUCSHON
BUDD
BURGESS
BURR
CAPITO
checking local file
CARDENAS
CARDIN
CASSIDY
CHABOT
CHENEY
checking local file
CICILLINE
COLE
COLLINS
CONAWAY
CORKER
CORNYN
CORREA
checking local file
COSTA
COURTNEY
CRAMER
CULBERSON
CURTIS
checking local file
DAINES
DAVIDSON
checking local file
DEMINGS
checking local file
DEUTCH
DONNELLY
DONOVAN
DUNN
checking local file
DURBIN
EMMER
ERNST
ESPAILLAT
checking local file
FITZPATRICK
FLAKE
FOXX
GABBARD
GALLAGHER
checking local file
GARAMENDI
GARDNER
checking local file
GARRETT
GIANFORTE
checking local file
GOHMERT
GOMEZ
checking local file
GOODLATTE
GOTTHEIMER
checking local file
GRIJALVA
GUTIERREZ
HASTINGS
HATCH
HECK
HEINRICH
HILL
HIMES
HIRONO
HOEVEN
HOYER
HUFFMAN
INHOFE
ISAKSON
JAYAPAL
checking local file
JEFFRIES
KAINE
KAPTUR
KATKO
KEATING
KENNEDY
KIHUEN
checking local file
KILDEE
KLOBUCHAR

speaker_id,first_name,last_name,type,party,state,district,bio_guide_id,congress_id
1695,Lamar,ALEXANDER,SENATOR,R,TN,N/A,A000360,114
2239,Rick,ALLEN,REPRESENTATIVE,R,GA,12,A000372,114
2029,Justin,AMASH,REPRESENTATIVE,R,MI,3,A000367,114
2270,Brian,BABIN,REPRESENTATIVE,R,TX,36,B001291,114
1881,John,BARRASSO,SENATOR,R,WY,N/A,B001261,114


speech_id,speaker_id,proceeding_id,topic_id,word_count,speech_text,file_name
48,CORNYN,proceeding_id,topic-id,2309,"Mr.President, many recall that Christmas came a little e ...",CREC-2018-01-03-pt2-PgS10.txt
49,DURBIN,proceeding_id,topic-id,25,"Mr.President, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS10.txt
51,DURBIN,proceeding_id,topic-id,893,"Mr.President, on September 5 of last year, Attorney Gene ...",CREC-2018-01-03-pt2-PgS11.txt
52,WHITEHOUSE,proceeding_id,topic-id,23,"Mr.President, I ask unanimous consent that the order for ...",CREC-2018-01-03-pt2-PgS11.txt
54,WHITEHOUSE,proceeding_id,topic-id,2035,"Thank you, Mr.President, and happy new year to you. For ...",CREC-2018-01-03-pt2-PgS13.txt


In [239]:
#populating speaker_id column of the speeches table.
# newcol = make_array()
# names = speeches.column('speaker_id')
# for i in np.arange(len(names)):
#     newcol = np.append(speakers.where('last_name', are.equal_to(names[i])).column('speaker_id').item(0), newcol)
# copy_speeches = speeches.with_column('num_speaker_id', newcol)
# copy_speeches.show(20)

In [240]:
names = speakers.column('last_name')
ids = speakers.column('speaker_id')
name_to_id = dict(zip(names, ids))
name_to_id['CAPITO'] = 1676

In [241]:
newcol = make_array()
for name in speeches.sort('speaker_id').column('speaker_id'):
    newcol = np.append(name_to_id[name], newcol)
speeches = speeches.sort('speaker_id').drop('speaker_id').with_column('speaker_id', np.flip(newcol, 0))

speech_id,proceeding_id,topic_id,word_count,speech_text,file_name,speaker_id
526,proceeding_id,topic-id,37,"I thank the distinguished Presiding Officer, the Senator ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695
527,proceeding_id,topic-id,410,"Mr.President, on December 1, 2016, Judge Todd Campbell s ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695
528,proceeding_id,topic-id,36,"Mr.President, I ask for the yeas and nays. The PRESIDIN ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695
977,proceeding_id,topic-id,222,"Mr.Speaker, I rise today to thank President Trump for sh ...",CREC-2018-01-11-pt1-PgH163-2.txt,2239
907,proceeding_id,topic-id,6963,"Mr.Speaker, I have an amendment at the desk. The SPEAKE ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029
908,proceeding_id,topic-id,153,"Mr.Speaker, I yield myself such time as I may consume. ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029
911,proceeding_id,topic-id,11,"Mr.Speaker, I yield 1 minute to the gentlewoman from Cal ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029
915,proceeding_id,topic-id,24,"Mr.Speaker, my amendment protects the rights of American ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029
918,proceeding_id,topic-id,12,"Mr.Speaker, I yield 30 seconds to the gentleman from New ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029
922,proceeding_id,topic-id,12,"Mr.Speaker, I yield 30 seconds to the gentleman from Cal ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029


In [249]:
title_column = make_array()
year_column = make_array()
month_column = make_array()
day_column = make_array()
for file_name in speeches.column('file_name'):
    title_column = np.append(find_title(file_name), title_column)
    year, month, day = sep_date_from_file(file_name)
    year_column = np.append(year, year_column)
    month_column = np.append(month, month_column)
    day_column = np.append(day, day_column)

    title_column = np.flip(title_column, 0)
year_column = np.flip(year_column, 0)
mont_column = np.flip(month_column, 0)
day_column = np.flip(day_column, 0)


In [250]:
speeches = speeches.drop('proceeding_id')
speeches = speeches.with_columns('session_title', title_column, 'year', year_column, 'month', month_column, 'day', day_column)

In [256]:
speeches

speech_id,topic_id,word_count,speech_text,file_name,speaker_id,session_title,year,month,day
526,topic-id,37,"I thank the distinguished Presiding Officer, the Senator ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695,EXECUTIVE CALENDAR--Continued,2018,01,09
527,topic-id,410,"Mr.President, on December 1, 2016, Judge Todd Campbell s ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695,EXECUTIVE CALENDAR--Continued,2018,01,09
528,topic-id,36,"Mr.President, I ask for the yeas and nays. The PRESIDIN ...",CREC-2018-01-09-pt1-PgS88-2.txt,1695,EXECUTIVE CALENDAR--Continued,2018,01,09
977,topic-id,222,"Mr.Speaker, I rise today to thank President Trump for sh ...",CREC-2018-01-11-pt1-PgH163-2.txt,2239,SPOTLIGHT ON RURAL AMERICA,2018,01,11
907,topic-id,6963,"Mr.Speaker, I have an amendment at the desk. The SPEAKE ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11
908,topic-id,153,"Mr.Speaker, I yield myself such time as I may consume. ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11
911,topic-id,11,"Mr.Speaker, I yield 1 minute to the gentlewoman from Cal ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11
915,topic-id,24,"Mr.Speaker, my amendment protects the rights of American ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11
918,topic-id,12,"Mr.Speaker, I yield 30 seconds to the gentleman from New ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11
922,topic-id,12,"Mr.Speaker, I yield 30 seconds to the gentleman from Cal ...",CREC-2018-01-11-pt1-PgH137-3.txt,2029,RAPID DNA ACT OF 2017,2018,01,11


In [259]:
speakers.where('district', are.equal_to('N/A')).where('type', are.equal_to('REPRESENTATIVE'))

speaker_id,first_name,last_name,type,party,state,district,bio_guide_id,congress_id
2307,Andy,BIGGS,REPRESENTATIVE,R,AZ,N/A,B001302,115
1723,Madeleine,BORDALLO,REPRESENTATIVE,D,GU,N/A,B001245,115
1742,Tom,CORREA,REPRESENTATIVE,R,OK,N/A,C001053,115
2363,John,CURTIS,REPRESENTATIVE,R,UT,N/A,C001114,115
2320,Val,DEMINGS,REPRESENTATIVE,D,FL,N/A,D000627,115
2315,Neal,DUNN,REPRESENTATIVE,R,FL,N/A,D000628,115
2342,Adriano,ESPAILLAT,REPRESENTATIVE,D,NY,N/A,E000297,115
2355,Mike,GALLAGHER,REPRESENTATIVE,R,WI,N/A,G000579,115
2359,Greg,GIANFORTE,REPRESENTATIVE,R,MT,N/A,G000584,115
2362,Jimmy,GOMEZ,REPRESENTATIVE,D,CA,N/A,G000585,115


168